## Module for building Henderson's Mixed Model Equations for single trait including marker  and polygenic effect



### <font color="red"> Data</font>

In [1]:
include("../src/MMEModule.jl")

MMEModule

In [2]:
using DataFrames
using QTLDatasets
using MMEModule

In [3]:
phenofile = QTLDatasets.dataset("testMME","simple.txt")
genofile = QTLDatasets.dataset("testMME","genotype.txt")
pedfile   = QTLDatasets.dataset("testMME","pedigree.txt")

"/Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMME/pedigree.txt"

In [4]:
phenotype = readtable(phenofile,separator = ',',header=true)

,Animal,Age,y
1,S1,1,-0.92
2,D1,2,-1.05
3,O1,3,-0.92
4,O3,2,1.2


In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1,0,1,1,1
D1,2,0,2,2,1
O1,1,2,0,1,0
O3,0,0,2,1,1


### <font color="red"> Run module</font>

In [6]:
varRes = 1.0
varGen = 2.5
markerProp = 0.8
MVarGen= varGen*markerProp
PVarGen= varGen*(1-markerProp);

In [7]:
ped = PedModule.mkPed(pedfile);

In [8]:
varRes=1.0
mme = MMEModule.build_model("y = intercept + Age + Animal",varRes)
MMEModule.set_covariate(mme,"Age")
MMEModule.set_random(mme,"Animal",ped,PVarGen)

### <font color="red"> work with markers</font>

In [9]:
MMEModule.add_markers(mme,genofile,MVarGen,separator=',');

The delimiters in file /Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMME/genotype.txt is ,  .


### <font color="red"> run MCMC</font>

In [10]:
output=MMEModule.runMCMC(mme,phenotype,chain_length=50000,
                         printout_frequency=5000,methods="BayesC",Pi=0.8,
                         output_marker_effects_frequency=1000)

at sample: 5000 with meanVare: 1.1682561655474932
at sample: 10000 with meanVare: 1.146805407838937
at sample: 15000 with meanVare: 1.1610481989029204
at sample: 20000 with meanVare: 1.178576981213072
at sample: 25000 with meanVare: 1.1733052501469943
at sample: 30000 with meanVare: 1.1622525472365408
at sample: 35000 with meanVare: 1.161035043118952
at sample: 40000 with meanVare: 1.1501234876578255
at sample: 45000 with meanVare: 1.1509511084521353
at sample: 50000 with meanVare: 1.1523109941628331


Dict{Any,Any} with 4 entries:
  "Posterior Mean of Loca… => 7x2 Array{Any,2}:…
  "MCMC samples for resid… => [0.679684,0.58551,1.43908,0.705037,0.470015,0.865…
  "MCMC samples for genet… => 50000x1 Array{Float64,2}:…
  "Posterior Mean of Mark… => [-0.2987018723158025,-0.12395720134615784,0.09532…

In [11]:
keys(output)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "Posterior Mean of Location Parameters"
  "MCMC samples for residual variance"
  "MCMC samples for genetic var-cov parameters"
  "Posterior Mean of Marker Effects"

In [12]:
output["Posterior Mean of Location Parameters"]

7x2 Array{Any,2}:
 "intercept: intercept"  -0.936964   
 "Age: Age"               0.248339   
 "Animal: S1"             0.00309496 
 "Animal: D1"            -0.000620785
 "Animal: O1"            -0.0281503  
 "Animal: O3"             0.0970751  
 "Animal: O2"             0.00559842 

In [13]:
output["Posterior Mean of Marker Effects"]

5-element Array{Float64,1}:
 -0.298702 
 -0.123957 
  0.0953282
 -0.108071 
  0.154245 

In [14]:
output[ "MCMC samples for genetic var-cov parameters"]

50000x1 Array{Float64,2}:
 0.22142 
 0.311651
 0.127704
 0.348496
 0.217233
 0.159236
 0.362204
 0.195517
 0.104291
 0.197336
 0.159772
 0.227538
 0.132382
 ⋮       
 0.184224
 0.191131
 0.327721
 0.299434
 0.865846
 0.504255
 0.177754
 0.279193
 0.441899
 0.349182
 0.300953
 0.500673

In [15]:
output["MCMC samples for residual variance"]

50000-element Array{Float64,1}:
 0.679684
 0.58551 
 1.43908 
 0.705037
 0.470015
 0.865157
 1.4342  
 0.518495
 0.865078
 0.335365
 1.00218 
 0.693965
 3.5976  
 ⋮       
 0.997715
 0.709423
 3.75033 
 0.881523
 1.67993 
 0.892106
 1.1598  
 0.848082
 1.12722 
 0.8265  
 0.381788
 0.294895